In [80]:
import numpy as np
import pandas as pd
import networkx as nx
import json
from collections import Counter

In [81]:
#nainštalujte
#!pip install isbnlib

zatial len cast siete, cele mi to zozralo calu ram

pocitanie matic celkovej podobnosti je implementovane, ale pre celu siet bude asi narocne si vsetko pamatat naraz, oplati sa teda pocitat len 'stlpec' pre kazdeho uzivatela, ked sa na neho budeme pytat

celkovo bude zaujiamve sa pytat aj na podobne knihy, ako aj podobnych uzivatelov
pre knihy -> "odporucanie, kupte si aj toto pri knihe"
pre pouzivatelov -> podobneho pouzivatela -> odporucime knihu, ktoru on cital, a my nie

na knihy z malych komponentov sa asi nebude mat zmysel pytat, giganticky komponent bude zaujimavsi

In [82]:
from isbnlib import meta
from isbnlib.registry import bibformatters

In [83]:
# df = pd.read_csv('data/rec-amz-Books.edges', header=None, names=['ID_reviewer', 'ISBN', 'Stars', 'Date'])

In [ ]:
def subgraphByListOfValues(values: pd.Series, column: str, df: pd.DataFrame) -> pd.DataFrame:
    return df[np.isin(df[column], values)]

In [84]:
def randomSubgraph(n:int, column: str, df: pd.DataFrame, weighed: bool = True, seed: int = 3) -> pd.DataFrame:
    np.random.seed(seed)
    values, counts = np.unique(df[column], return_counts=True)
    return subgraphByListOfValues(np.random.choice(values, n, replace=False, p=(counts / sum(counts)) if weighed else None), column, df)

In [85]:
# subdf = randomSubgraph(150, 'ISBN', df, seed=3)

In [86]:
# '0439023483' in subdf['ISBN']

In [87]:
# subdf


In [88]:
# v, c = np.unique(subdf['ISBN'] , return_counts=True)
# v[c.argmax()]

In [89]:
class RecommendationEngine:
    def __init__(self, filename: str):
        self.df = pd.read_csv(filename, header=None, names=['ID_reviewer', 'ISBN','Stars','Date'])
        self.subdf = randomSubgraph(200, 'ISBN', self.df)
        self.SM = None
        self.SMC = None
        self.G: nx.classes.graph.Graph = nx.from_pandas_edgelist(self.subdf, source='ID_reviewer', target='ISBN', edge_attr='Stars')

    def filterNodes(self, users: bool) -> list[str]:
        return list(filter(lambda x: (x[0] == 'A') if users else (x[0] != 'A'), self.G.nodes()))

    def similarityMatrix(self, users: bool, save: bool=False) -> pd.DataFrame:
        # if self.SM is not None:
        #     return self.SM
        # A = nx.to_numpy_array(re.G)
        # SM = A @ A
        # if save:
        #     self.SM = SM
        # return SM
        filterList = self.filterNodes(users)
        if self.SM is not None:
            return self.SM.loc[filterList,filterList]
        order = self.G.nodes()
        A = nx.to_numpy_array(self.G, nodelist=order)
        product = A @ A
        np.fill_diagonal(product, 0)
        SM = pd.DataFrame(product, columns = order)
        SM.index = order
        if save:
            self.SM = SM
        return SM.loc[filterList,filterList]

    def similarityMatrixCosine(self, users: bool, save: bool=False) -> np.ndarray:
        filterList = self.filterNodes(users)
        if self.SMC is not None:
            return self.SMC.loc[filterList,filterList]
        SM = self.similarityMatrix(users)
        norms = (np.linalg.norm(SM, axis=1))
        SMC = SM / norms[:, None] / norms[None, :]
        np.nan_to_num(SMC, copy=False)
        if save:
            self.SMC = SMC
        return SMC.loc[filterList,filterList]


    def getMetaFromISBN(self,isbn:str) -> json:
      SERVICE = "goob"
      format_json = bibformatters["json"]
      meta_data = (format_json(meta(isbn, SERVICE)))
      #print(meta_data)
      return meta_data

    def fromJsonToDict(self,json_file:json) -> dict:
      return json.loads(json_file)

    def getDf(self)->pd.DataFrame:
      return self.subdf

    def getBooksByUser(self,df:pd.DataFrame,id:str) ->list: #(list of ISBN for user)
      df = df.loc[df['ID_reviewer']==id, 'ISBN'].values
      return df

    def getBooks(self,df:pd.DataFrame,id1:str,id2:str,type_:str) ->list: #same - same books, diff - different books
      b1 = self.getBooksByUser(data,id1)
      b2 = self.getBooksByUser(data,id2)
      if type_ == "same":
        return list(set(b1).intersection(set(b2)))
      elif type_ == "diff1":
        return list(set(b2) - set(b1)) #vsetky knihy od id2, ktore id1 necital
      elif type_ == "diff2":
        return list(set(b1) - set(b2)) #vsetky knihy od id1, ktore id2 necital

    def getSimilarBooks(self,df:pd.DataFrame,id1:str,id2:str,type_:str) ->list:  #type ==(year, publisher)
        if type_ == "year": #odporucame id1 knihy od uzivatela id2 na zaklade roku vydania knihy
          books_s = self.getBooks(df,id1,id2,"same")
          #print(books_s)
          years = list()
          for i in range(len(books_s)): #zoberieme vseetky roky a pozrieme sa aky najviac prevlada -> zoberiem okolie (-5,5) a to odporucime
            pom = self.getMetaFromISBN(books_s[i])
            if pom != None: #nenaslo ISBN
              years.append(self.fromJsonToDict(pom)["year"])
          most_common_era = Counter(years).most_common(1) #rok,pocet_opakovani
          most_common_era = [int(most_common_era[0][0])-5,int(most_common_era[0][0])+5]
          #print(most_common_era)
          books_d = self.getBooks(df,id1,id2,"diff1")
          recommendation_by_year = list()
          for i in range(len(books_d)): #zistime info o vsetkych kniha id2 a vyberieme nazvy tych ktorych vydania su medzi most_common_era
            pom = self.getMetaFromISBN(books_d[i])
            if pom != None: #nenaslo ISBN
              if most_common_era[0] <= (int(self.fromJsonToDict(pom)["year"])) <= most_common_era[1]:
                title = self.fromJsonToDict(pom)["title"]
                year = self.fromJsonToDict(pom)["year"]
                authors = list()
                for i in range(len(self.fromJsonToDict(pom)["author"])):  #ak mame viac autorov
                  authors.append(self.fromJsonToDict(pom)["author"][i]['name'])
                recommendation_by_year.append([title,year, authors])
                #print(recommendation_by_year)
          if recommendation_by_year == []:
            recommendation_by_year = [("Nenasli sa ziadne knihy z rokov:",most_common_era[0],"-",most_common_era[1])]
          return recommendation_by_year

        elif type_ == "publisher": #odporucame id1 knihy od uzivatela id2, vyberame oblubeneho vydavatelstvo
           books_s = self.getBooks(df,id1,id2,"same")
           publishers = list()
           for i in range(len(books_s)): #zoberieme vseetky vydavatelstva -> most_common
              pom = self.getMetaFromISBN(books_s[i])
              if pom != None: #nenaslo ISBN
                publishers.append(self.fromJsonToDict(pom)["publisher"])
           most_common_publisher = Counter(publishers).most_common(1) #vydavatelstvo,pocet_opakovani
           print(most_common_publisher)
           books_d = self.getBooks(df,id1,id2,"diff1")
           recommendation_by_publisher = list()
           for i in range(len(books_d)): #zistime info o vsetkych kniha id2 a vyberieme nazvy tych ktorych vydania su od most_common_publishe
              pom = self.getMetaFromISBN(books_d[i])
              if pom != None: #nenaslo ISBN
                if most_common_publisher[0] == (self.fromJsonToDict(pom)["publisher"]):
                  title = self.fromJsonToDict(pom)["title"]
                  year = self.fromJsonToDict(pom)["year"]
                  authors = list()
                  for i in range(len(self.fromJsonToDict(pom)["author"])): #ak mame viac autorov
                    authors.append(self.fromJsonToDict(pom)["author"][i]['name'])
                  recommendation_by_publisher.append([title,year, authors])
                #print(recommendation_by_year)
           if recommendation_by_publisher == []:
              recommendation_by_publisher = ["Nenasli sa ziadne knihy od oblubeneho vydavatela"]
           return recommendation_by_publisher


In [90]:
re = RecommendationEngine('data/rec-amz-Books.edges')

In [91]:
'0439023483' in re.subdf['ISBN']

False

In [92]:
nx.is_connected(re.G)

False

In [93]:
data = re.getDf()

In [94]:
re.subdf

,ID_reviewer,ISBN,Stars,Date
539610,A300KQBH76WJSK,0061231134,4.0,1374883200
539611,AAEPR16TFJU06,0061231134,4.0,1109203200
539612,A1TRH2JALWQ6I0,0061231134,3.0,1328745600
539613,A2HA1CNG8SKTUN,0061231134,1.0,1269216000
539614,A2USQ41MEPJXXL,0061231134,3.0,1138838400
...,...,...,...,...
21967025,A1F7NX5BA22PZD,B00H5SE1TG,4.0,1388880000
22010923,A3J0W78JCYNOFA,B00HG1X9YA,5.0,1405987200
22010924,A1GWENX3EN39VV,B00HG1X9YA,4.0,1388707200
22010925,AN0N05A9LIJEQ,B00HG1X9YA,5.0,1389830400


In [ ]:
re.similarityMatrix(users=True)

In [ ]:
re.similarityMatrix(users=False)

In [ ]:
re.similarityMatrixCosine(users=True)

In [ ]:
re.similarityMatrixCosine(users=False)

In [ ]:
meta_json_book = re.getMetaFromISBN("0000013714")
list_meta = re.fromJsonToDict(meta_json_book)
print(list_meta)
print(list_meta["publisher"])

In [ ]:
print(re.getBooksByUser(data,"A1KP12IVP6LAGL"))
books_users = re.getBooks(data,"A1KP12IVP6LAGL","AG7R3MMF8QLDT","same")
print("same:" ,books_users)
books_users = re.getBooks(data,"A1KP12IVP6LAGL","AG7R3MMF8QLDT","diff1")
print("diff1:" ,books_users)
books_users = re.getBooks(data,"A1KP12IVP6LAGL","AG7R3MMF8QLDT","diff2")
print("diff2:" ,books_users)

In [ ]:
print("Recommendation by year: ")
books_recomendation = re.getSimilarBooks(data,"A1KP12IVP6LAGL","AG7R3MMF8QLDT","year")
print(books_recomendation)
print("Recommendation by publisher: ")
books_recomendation = re.getSimilarBooks(data,"A1KP12IVP6LAGL","AG7R3MMF8QLDT","publisher")
print(books_recomendation)